# core

> Mixins, ProjectIO, and PIO for centralized path management.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
from dataclasses import dataclass
from pathlib import Path
from types import ModuleType
from datetime import datetime
from typing import Callable, Literal, Mapping, Sequence, Union, Iterable, Optional, Dict, List, Any
import importlib.resources as resources
import contextlib

from projio.funcs import (
    normalize_path,
    ensure_extension,
    format_datestamp,
    parse_datestamp as parse_ds_func,
    build_tree,
    render_gitignore,
    resolve_template,
    TemplateSpec,
)

In [ ]:
#| export
StrPath = Union[str, Path]
Kind = Literal["outputs", "logs", "checkpoints", "tensorboard", "cache", "data", "resources"]
DatePlacement = Literal["dirs", "files", "both", "none"]

# Built-in templates - defined early so mixins can reference them
# Lightning-specific templates (registered by LightningMixin)
LIGHTNING_TEMPLATES: Dict[str, TemplateSpec] = {
    "checkpoints": TemplateSpec(
        name="checkpoints",
        base=lambda io: io.checkpoints,
        pattern=["{run}", "{model}.ckpt"]
    ),
    "tensorboard": TemplateSpec(
        name="tensorboard",
        base=lambda io: io.tensorboard,
        pattern=["{run}"]
    ),
}

# Core/non-Lightning templates (registered by TemplateMixin)
CORE_TEMPLATES: Dict[str, TemplateSpec] = {
    "filtered_matrix": TemplateSpec(
        name="filtered_matrix",
        base=lambda io: io.outputs,
        pattern={
            "barcodes": "barcodes.tsv.gz",
            "matrix": "matrix.mtx",
            "features": "features.tsv.gz"
        }
    ),
    "notebook_outputs": TemplateSpec(
        name="notebook_outputs",
        base=lambda io: io.outputs,
        pattern=["{notebook}", "{run}"]
    ),
}

## ProducerRecord

Tracks which script/notebook produced a file.

In [ ]:
#| export
@dataclass
class ProducerRecord:
    """Record of a file's producer.
    
    Attributes:
        target: Path to the produced file.
        producer: Path to the script/notebook that produced it.
        kind: Optional kind/type of output.
    """
    target: Path
    producer: Path
    kind: Optional[str] = None

## Mixins

Mixins provide modular functionality grouped by concern.

### RootMixin

Handles root/iroot/oroot cascade, setters, cwd/cwp resolution, auto_create handling.

In [ ]:
#| export
class RootMixin:
    """Mixin for root path management and cascade behavior."""
    
    def __init__(self, *, root: StrPath | None = None, iroot: StrPath | None = None,
                 oroot: StrPath | None = None, auto_create: bool = True, **kwargs):
        super().__init__(**kwargs)
        self.auto_create = auto_create
        self.stored_cwd = Path.cwd()
        self.stored_cwp = Path.cwd()
        
        # Track which roots were explicitly set by user
        self.user_set_iroot = iroot is not None
        self.user_set_oroot = oroot is not None
        
        # Initialize roots
        self.stored_root = normalize_path(root, self.stored_cwp) if root is not None else self.stored_cwp
        self.stored_iroot = normalize_path(iroot, self.stored_root) if iroot is not None else self.stored_root
        self.stored_oroot = normalize_path(oroot, self.stored_root) if oroot is not None else self.stored_root
    
    @property
    def cwd(self) -> Path:
        """Current working directory (captured at import time)."""
        return self.stored_cwd
    
    @property
    def cwp(self) -> Path:
        """Current working project directory."""
        return self.stored_cwp
    
    @cwp.setter
    def cwp(self, value: StrPath):
        """Set current working project."""
        self.stored_cwp = normalize_path(value, self.stored_cwd)
    
    @property
    def root(self) -> Path:
        """Shared base for outputs; cascades to iroot/oroot."""
        return self.stored_root
    
    @root.setter
    def root(self, value: StrPath):
        """Set root and cascade to iroot/oroot if not user-set."""
        if not isinstance(value, (str, Path)):
            raise TypeError(f"root must be str or Path, got {type(value).__name__}")
        self.stored_root = normalize_path(value, self.stored_cwp)
        # Cascade to iroot/oroot only if not explicitly set by user
        if not self.user_set_iroot:
            self.stored_iroot = self.stored_root
        if not self.user_set_oroot:
            self.stored_oroot = self.stored_root
    
    @property
    def iroot(self) -> Path:
        """Input/data root."""
        return self.stored_iroot
    
    @iroot.setter
    def iroot(self, value: StrPath):
        """Set input root."""
        if not isinstance(value, (str, Path)):
            raise TypeError(f"iroot must be str or Path, got {type(value).__name__}")
        self.stored_iroot = normalize_path(value, self.stored_root)
        self.user_set_iroot = True
    
    @property
    def oroot(self) -> Path:
        """Output root."""
        return self.stored_oroot
    
    @oroot.setter
    def oroot(self, value: StrPath):
        """Set output root."""
        if not isinstance(value, (str, Path)):
            raise TypeError(f"oroot must be str or Path, got {type(value).__name__}")
        self.stored_oroot = normalize_path(value, self.stored_root)
        self.user_set_oroot = True
    
    @property
    def inputs(self) -> Path:
        """Alias for iroot."""
        return self.ensure_dir(self.stored_iroot)
    
    @property
    def outputs(self) -> Path:
        """Alias for oroot."""
        return self.ensure_dir(self.stored_oroot)
    
    @property
    def data_dir(self) -> Path:
        """Data directory under inputs."""
        return self.ensure_dir(self.stored_iroot / "data")
    
    @property
    def downloads(self) -> Path:
        """Downloads directory under inputs."""
        return self.ensure_dir(self.stored_iroot / "downloads")
    
    @property
    def cache(self) -> Path:
        """Cache directory under outputs."""
        return self.ensure_dir(self.stored_oroot / "cache")
    
    @property
    def logs(self) -> Path:
        """Logs directory under outputs."""
        return self.ensure_dir(self.stored_oroot / "logs")
    
    def ensure_dir(self, path: Path) -> Path:
        """Create directory if auto_create and not dry_run."""
        if self.auto_create and not getattr(self, "dry_run", False):
            path.mkdir(parents=True, exist_ok=True)
        return path

### DatestampMixin

Handles datestamp formatting, parsing, and placement rules.

In [ ]:
#| export
class DatestampMixin:
    """Mixin for datestamp behavior."""
    
    def __init__(self, *, use_datestamp: bool = True,
                 datestamp_format: str = "%Y_%m_%d",
                 datestamp_in: DatePlacement = "dirs", **kwargs):
        super().__init__(**kwargs)
        self.use_datestamp = use_datestamp
        self.datestamp_format = datestamp_format
        self.datestamp_in = datestamp_in
    
    def datestamp_value(self, timestamp: datetime | None = None) -> str:
        """Get formatted datestamp string.
        
        Parameters:
            timestamp: Specific datetime; uses now() if None.
            
        Returns:
            Formatted datestamp string.
        """
        return format_datestamp(timestamp, self.datestamp_format)
    
    def parse_datestamp(self, text: str) -> datetime:
        """Parse datestamp string to datetime.
        
        Parameters:
            text: Datestamp string to parse.
            
        Returns:
            Parsed datetime.
            
        Raises:
            ValueError: If text doesn't match datestamp_format.
        """
        return parse_ds_func(text, self.datestamp_format)

### TemplateMixin

Handles template registration and resolution for non-Lightning templates.

In [ ]:
#| export
class TemplateMixin:
    """Mixin for template registration and path building."""
    
    def __init__(self, *, templates: Dict[str, TemplateSpec] | None = None, **kwargs):
        self.templates: Dict[str, TemplateSpec] = templates.copy() if templates else {}
        super().__init__(**kwargs)
        self.register_core_templates()
    
    def register_template(self, spec: TemplateSpec) -> TemplateSpec:
        """Register a template specification.
        
        Parameters:
            spec: Template specification to register.
            
        Returns:
            The registered spec.
        """
        self.templates[spec.name] = spec
        return spec
    
    def template_path(self, name: str, variant: str | None = None,
                      datestamp: bool | None = None,
                      timestamp: datetime | None = None, **fmt) -> Path | Dict[str, Path]:
        """Resolve a template to a concrete path.
        
        Parameters:
            name: Registered template name.
            variant: Optional variant (run/model name).
            datestamp: Override datestamp behavior.
            timestamp: Specific timestamp for datestamp.
            **fmt: Format placeholders for pattern.
            
        Returns:
            Path or dict of paths depending on template pattern.
            
        Raises:
            ValueError: If template name not found.
        """
        if name not in self.templates:
            raise ValueError(f"Unknown template: {name}. Available: {list(self.templates.keys())}")
        spec = self.templates[name]
        return resolve_template(spec, self, variant, fmt, datestamp, timestamp)
    
    def register_core_templates(self):
        """Register built-in non-Lightning templates."""
        for spec in CORE_TEMPLATES.values():
            self.register_template(spec)

### LightningMixin

Handles Lightning-specific paths and helpers.

In [ ]:
#| export
class LightningMixin:
    """Mixin for Lightning-aware paths and helpers."""
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.register_lightning_templates()
    
    @property
    def lightning_root(self) -> Path:
        """Root for Lightning artifacts."""
        return self.ensure_dir(self.outputs / "lightning")
    
    @property
    def checkpoints(self) -> Path:
        """Checkpoints directory."""
        return self.ensure_dir(self.lightning_root / "checkpoints")
    
    @property
    def tensorboard(self) -> Path:
        """TensorBoard logs directory."""
        return self.ensure_dir(self.lightning_root / "tensorboard")
    
    @property
    def lightning_logs(self) -> Path:
        """Lightning logs directory."""
        return self.ensure_dir(self.lightning_root / "logs")
    
    def checkpoint_path(self, name: str, ext: str = ".ckpt",
                        run: str | None = None,
                        datestamp: bool | None = None) -> Path:
        """Build checkpoint file path.
        
        Parameters:
            name: Checkpoint name (without extension).
            ext: File extension (default .ckpt).
            run: Optional run subdirectory.
            datestamp: Override datestamp behavior.
            
        Returns:
            Full path to checkpoint file.
        """
        if run and ("/" in run or "\\" in run):
            raise ValueError(f"run cannot contain path separators: {run}")
        
        base = self.checkpoints
        parts: list[str] = []
        ds = datestamp if datestamp is not None else self.use_datestamp
        
        if ds and self.datestamp_in in ("dirs", "both"):
            parts.append(self.datestamp_value())
        if run:
            parts.append(run)
        
        filename = ensure_extension(name, ext)
        if ds and self.datestamp_in in ("files", "both"):
            filename = f"{self.datestamp_value()}__{filename}"
        
        target = base.joinpath(*parts, filename)
        if self.auto_create and not self.dry_run:
            target.parent.mkdir(parents=True, exist_ok=True)
        return target
    
    def log_path(self, name: str, ext: str = ".log",
                 run: str | None = None,
                 datestamp: bool | None = None) -> Path:
        """Build log file path.
        
        Parameters:
            name: Log name (without extension).
            ext: File extension (default .log).
            run: Optional run subdirectory.
            datestamp: Override datestamp behavior.
            
        Returns:
            Full path to log file.
        """
        if run and ("/" in run or "\\" in run):
            raise ValueError(f"run cannot contain path separators: {run}")
        
        base = self.logs
        parts: list[str] = []
        ds = datestamp if datestamp is not None else self.use_datestamp
        
        if ds and self.datestamp_in in ("dirs", "both"):
            parts.append(self.datestamp_value())
        if run:
            parts.append(run)
        
        filename = ensure_extension(name, ext)
        if ds and self.datestamp_in in ("files", "both"):
            filename = f"{self.datestamp_value()}__{filename}"
        
        target = base.joinpath(*parts, filename)
        if self.auto_create and not self.dry_run:
            target.parent.mkdir(parents=True, exist_ok=True)
        return target
    
    def tensorboard_run(self, run: str | None = None,
                        datestamp: bool | None = None) -> Path:
        """Build TensorBoard run directory path.
        
        Parameters:
            run: Optional run name.
            datestamp: Override datestamp behavior.
            
        Returns:
            Path to TensorBoard run directory.
        """
        if run and ("/" in run or "\\" in run):
            raise ValueError(f"run cannot contain path separators: {run}")
        
        base = self.tensorboard
        parts: list[str] = []
        ds = datestamp if datestamp is not None else self.use_datestamp
        
        if ds and self.datestamp_in in ("dirs", "both"):
            parts.append(self.datestamp_value())
        if run:
            parts.append(run)
        
        target = base.joinpath(*parts)
        if self.auto_create and not self.dry_run:
            target.mkdir(parents=True, exist_ok=True)
        return target
    
    def register_lightning_templates(self):
        """Register built-in Lightning templates."""
        for spec in LIGHTNING_TEMPLATES.values():
            self.register_template(spec)

### GitignoreMixin

Handles .gitignore integration.

In [ ]:
#| export
class GitignoreMixin:
    """Mixin for gitignore management."""
    
    def __init__(self, *, gitignore: bool | StrPath | None = ".gitignore", **kwargs):
        super().__init__(**kwargs)
        self.gitignore = gitignore
    
    def ensure_gitignored(self, *kinds: Kind) -> None:
        """Ensure paths for given kinds are in .gitignore.
        
        Parameters:
            *kinds: Path kinds to add to gitignore.
        """
        if not self.gitignore:
            return
        entries = []
        for kind in kinds:
            path = self.path_for(kind, create=False)
            try:
                rel = path.relative_to(self.cwp)
                entries.append(str(rel) + "/")
            except ValueError:
                entries.append(str(path) + "/")
        self.append_gitignore(entries)
    
    def append_gitignore(self, entries: Iterable[str]) -> None:
        """Append entries to .gitignore file.
        
        Parameters:
            entries: Lines to add to gitignore.
        """
        if not self.gitignore:
            return
        
        if self.gitignore is True:
            gi_path = self.cwp / ".gitignore"
        else:
            gi_path = normalize_path(self.gitignore, self.cwp)
        
        text = gi_path.read_text() if gi_path.exists() else ""
        new_text = render_gitignore(text, entries)
        
        if new_text != text and not self.dry_run:
            gi_path.parent.mkdir(parents=True, exist_ok=True)
            gi_path.write_text(new_text)

### TreeMixin

Handles ASCII tree rendering.

In [ ]:
#| export
class TreeMixin:
    """Mixin for directory tree display."""
    
    def tree(self, path: StrPath | None = None,
             max_depth: int = 4, files: bool = False) -> str:
        """Render ASCII directory tree.
        
        Parameters:
            path: Root path for tree (default: outputs).
            max_depth: Maximum depth to display.
            files: Include files (not just directories).
            
        Returns:
            ASCII tree string.
        """
        path = Path(path) if path else self.outputs
        return build_tree(path, max_depth=max_depth, files=files)

### ProducerMixin

Tracks which scripts/notebooks produced which files.

In [ ]:
#| export
class ProducerMixin:
    """Mixin for producer tracking."""
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.producers: list[ProducerRecord] = []
    
    def track_producer(self, target: StrPath, producer: StrPath, kind: str | None = None) -> None:
        """Record which producer created a target file.
        
        Parameters:
            target: Path to produced file.
            producer: Path to producing script/notebook.
            kind: Optional kind/type of output.
        """
        self.producers.append(ProducerRecord(
            target=normalize_path(target, self.cwp),
            producer=normalize_path(producer, self.cwp),
            kind=kind
        ))
    
    def producers_of(self, path: StrPath) -> List[ProducerRecord]:
        """Get all producer records for a target path.
        
        Parameters:
            path: Target path to look up.
            
        Returns:
            List of producer records for that target.
        """
        p = normalize_path(path, self.cwp)
        return [rec for rec in self.producers if rec.target == p]
    
    def outputs_of(self, producer: StrPath) -> List[ProducerRecord]:
        """Get all outputs created by a producer.
        
        Parameters:
            producer: Producer path to look up.
            
        Returns:
            List of producer records from that producer.
        """
        p = normalize_path(producer, self.cwp)
        return [rec for rec in self.producers if rec.producer == p]

### ContextMixin

Provides context manager and description utilities.

In [ ]:
#| export
class ContextMixin:
    """Mixin for context manager and introspection."""
    
    @contextlib.contextmanager
    def using(self, **overrides):
        """Temporarily override settings.
        
        Parameters:
            **overrides: Attribute names and temporary values.
            
        Yields:
            Self with overridden settings.
            
        Example:
            with io.using(dry_run=True):
                io.checkpoint_path("test")  # no filesystem changes
        """
        old = {}
        for k, v in overrides.items():
            if not hasattr(self, k):
                raise AttributeError(f"Unknown attribute: {k}")
            old[k] = getattr(self, k)
            setattr(self, k, v)
        try:
            yield self
        finally:
            for k, v in old.items():
                setattr(self, k, v)
    
    def describe(self) -> dict:
        """Describe current configuration.
        
        Returns:
            Dict of configuration values (paths as strings).
        """
        desc = {
            "root": str(self.root),
            "iroot": str(self.iroot),
            "oroot": str(self.oroot),
            "inputs": str(self.stored_iroot),
            "outputs": str(self.stored_oroot),
            "cache": str(self.stored_oroot / "cache"),
            "logs": str(self.stored_oroot / "logs"),
            "lightning_root": str(self.stored_oroot / "lightning"),
            "checkpoints": str(self.stored_oroot / "lightning" / "checkpoints"),
            "tensorboard": str(self.stored_oroot / "lightning" / "tensorboard"),
            "auto_create": self.auto_create,
            "use_datestamp": self.use_datestamp,
            "datestamp_format": self.datestamp_format,
            "datestamp_in": self.datestamp_in,
            "dry_run": self.dry_run,
        }
        # Add resources if available
        if hasattr(self, "package"):
            desc["package"] = str(self.package) if self.package else None
        # Add producer count
        if hasattr(self, "producers"):
            desc["producer_records"] = len(self.producers)
        return desc
    
    def __repr__(self) -> str:
        return f"ProjectIO(root={self.root}, inputs={self.stored_iroot}, outputs={self.stored_oroot})"

## ProjectIO

Main class composing all mixins.

In [ ]:
#| export
class ProjectIO(RootMixin, DatestampMixin, TemplateMixin,
                LightningMixin, GitignoreMixin, TreeMixin,
                ProducerMixin, ContextMixin):
    """Global path manager with template and Lightning awareness.
    
    Composes mixins to provide comprehensive path management for Python packages.
    Inspired by scanpy.settings singleton-style API and pathlib ergonomics.
    
    Parameters:
        package: Package name or module for resource discovery.
        root: Shared base path (cascades to iroot/oroot).
        iroot: Input/data root (default: root).
        oroot: Output root (default: root).
        auto_create: Auto-create directories on access (default: True).
        use_datestamp: Include datestamps in paths (default: True).
        datestamp_format: strftime format (default: %Y_%m_%d).
        datestamp_in: Where to add datestamp: 'dirs', 'files', 'both', 'none'.
        dry_run: Prevent filesystem mutations (default: False).
        gitignore: Path to gitignore or True/False (default: '.gitignore').
    
    Example:
        >>> io = ProjectIO(root="/project", use_datestamp=True)
        >>> io.checkpoint_path("model", run="exp1")
        PosixPath('/project/lightning/checkpoints/2024_03_15/exp1/model.ckpt')
    """
    
    def __init__(
        self,
        package: str | ModuleType | None = None,
        root: StrPath | None = None,
        iroot: StrPath | None = None,
        oroot: StrPath | None = None,
        auto_create: bool = True,
        use_datestamp: bool = True,
        datestamp_format: str = "%Y_%m_%d",
        datestamp_in: DatePlacement = "dirs",
        dry_run: bool = False,
        gitignore: bool | StrPath | None = ".gitignore",
    ):
        self.package = package
        self.dry_run = dry_run
        self.pending_actions: list[str] = []  # for dry_run tracking
        
        super().__init__(
            root=root,
            iroot=iroot,
            oroot=oroot,
            auto_create=auto_create,
            use_datestamp=use_datestamp,
            datestamp_format=datestamp_format,
            datestamp_in=datestamp_in,
            gitignore=gitignore,
        )
    
    @property
    def resources(self) -> Path:
        """Package resources directory.
        
        Discovered via importlib.resources if package is set,
        otherwise falls back to cwp/resources.
        """
        if self.package is None:
            path = self.cwp / "resources"
        else:
            try:
                pkg = self.package if isinstance(self.package, str) else self.package.__name__
                path = Path(str(resources.files(pkg))) / "resources"
            except Exception:
                path = self.cwp / "resources"
        if self.auto_create and not self.dry_run:
            path.mkdir(parents=True, exist_ok=True)
        return path
    
    def resource_path(self, *parts: str, must_exist: bool = True,
                      create: bool = False) -> Path:
        """Get path to a resource file.
        
        Parameters:
            *parts: Path components under resources dir.
            must_exist: Raise if path doesn't exist (default: True).
            create: Create the path if missing.
            
        Returns:
            Full path to resource.
            
        Raises:
            FileNotFoundError: If must_exist and path doesn't exist.
        """
        path = self.resources.joinpath(*parts)
        if create and not self.dry_run:
            if path.suffix:
                path.parent.mkdir(parents=True, exist_ok=True)
            else:
                path.mkdir(parents=True, exist_ok=True)
        if must_exist and not path.exists():
            raise FileNotFoundError(f"Resource not found: {path}")
        return path
    
    def path_for(self, kind: Kind, name: str = "", ext: str | None = None,
                 subdir: str | Sequence[str] | None = None,
                 datestamp: bool | None = None,
                 timestamp: datetime | None = None,
                 create: bool | None = None) -> Path:
        """Build a path for a given kind.
        
        Parameters:
            kind: One of 'outputs', 'logs', 'checkpoints', 'tensorboard', 'cache', 'data', 'resources'.
            name: Optional filename.
            ext: Optional extension.
            subdir: Optional subdirectory or list of subdirs.
            datestamp: Override datestamp behavior.
            timestamp: Specific timestamp for datestamp.
            create: Override auto_create for this call.
            
        Returns:
            Constructed path.
            
        Raises:
            ValueError: If kind is unknown.
        """
        base_map = {
            "outputs": self.stored_oroot,
            "logs": self.stored_oroot / "logs",
            "checkpoints": self.stored_oroot / "lightning" / "checkpoints",
            "tensorboard": self.stored_oroot / "lightning" / "tensorboard",
            "cache": self.stored_oroot / "cache",
            "data": self.stored_iroot / "data",
            "resources": self.resources,
        }
        if kind not in base_map:
            raise ValueError(f"Unknown kind: {kind}. Valid: {list(base_map.keys())}")
        
        base = base_map[kind]
        parts: list[str] = []
        
        if subdir:
            if isinstance(subdir, str):
                parts.append(subdir)
            else:
                parts.extend(subdir)
        
        ds = datestamp if datestamp is not None else self.use_datestamp
        if ds and self.datestamp_in in ("dirs", "both"):
            parts.append(self.datestamp_value(timestamp))
        
        filename = name
        if ext:
            filename = ensure_extension(filename, ext)
        
        if filename:
            target = base.joinpath(*parts, filename)
            if ds and self.datestamp_in in ("files", "both"):
                target = target.with_name(f"{self.datestamp_value(timestamp)}__{target.name}")
        else:
            target = base.joinpath(*parts) if parts else base
        
        should_create = create if create is not None else self.auto_create
        if should_create and not self.dry_run:
            if target.suffix:
                target.parent.mkdir(parents=True, exist_ok=True)
            else:
                target.mkdir(parents=True, exist_ok=True)
        
        return target

## PIO - Singleton Proxy

Class-level proxy for convenient access without explicit instantiation.

In [ ]:
#| export
class PIOType(type):
    """Metaclass for PIO to enable class-level attribute forwarding."""
    stored_default: ProjectIO | None = None
    
    @property
    def default(cls) -> ProjectIO:
        """Lazily instantiated default ProjectIO."""
        if cls.stored_default is None:
            cls.stored_default = ProjectIO()
        return cls.stored_default
    
    @default.setter
    def default(cls, value: ProjectIO | None):
        cls.stored_default = value
    
    def __getattr__(cls, name: str):
        return getattr(cls.default, name)
    
    def __setattr__(cls, name: str, value):
        # Handle stored_default and default specially to avoid triggering lazy init
        if name == "stored_default" or name == "default":
            type.__setattr__(cls, "stored_default", value)
        else:
            setattr(cls.default, name, value)

In [ ]:
#| export
class PIO(metaclass=PIOType):
    """Class-level proxy to a global ProjectIO singleton.
    
    Enables convenient access without explicit instantiation:
    
        >>> PIO.cache  # returns Path
        >>> PIO.checkpoint_path("model")  # builds path
        >>> PIO.root = "/new/root"  # changes default root
    
    Can subclass to set package-specific defaults:
    
        >>> class Paths(PIO):
        ...     package = "mypackage"
    """
    pass

## Examples

## Examples

In [ ]:
# Example usage
import tempfile
with tempfile.TemporaryDirectory() as tmp:
    io = ProjectIO(root=tmp, auto_create=False, use_datestamp=False)
    print(f"Root: {io.root}")
    print(f"Outputs: {io.outputs}")
    print(f"Checkpoints: {io.checkpoints}")
    
    # With datestamp
    io2 = ProjectIO(root=tmp, auto_create=False, use_datestamp=True, datestamp_in="dirs")
    io2.datestamp_value = lambda ts=None: "2024_03_15"  # Mock for demo
    print(f"\nCheckpoint path: {io2.checkpoint_path('model', run='exp1')}")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()